⊕ [什么是所有权？ · Rust 程序设计语言（第二版） 简体中文版](https://kaisery.gitbooks.io/trpl-zh-cn/content/ch04-01-what-is-ownership.html)


In [2]:
let mut s = String::from("hello");

s.push_str(", world!"); // push_str() 在字符串后追加字面值

println!("{}", s); // 将打印 `hello, world!`

hello, world!


In [3]:
let s1 = String::from("hello");
let s2 = s1.clone();

println!("s1 = {}, s2 = {}", s1, s2);

s1 = hello, s2 = hello


## Copy trait 的特殊注解
Rust 有一个叫做 Copy trait 的特殊注解，可以用在类似整型这样的存储在栈上的类型上（第十章详细讲解 trait）。如果一个类型拥有 Copy trait，一个旧的变量在将其赋值给其他变量后仍然可用。Rust 不允许自身或其任何部分实现了 Drop trait 的类型使用 Copy trait。如果我们对其值离开作用域时需要特殊处理的类型使用 Copy 注解，将会出现一个编译时错误。要学习如何为你的类型增加 Copy 注解，请阅读附录 C 中的 “可派生的 trait”。

那么什么类型是 Copy 的呢？可以查看给定类型的文档来确认，不过作为一个通用的规则，任何简单标量值的组合可以是 Copy 的，不需要分配内存或某种形式资源的类型是 Copy 的。如下是一些 Copy 的类型：

- 所有整数类型，比如 u32。
- 布尔类型，bool，它的值是 true 和 false。
- 所有浮点数类型，比如 f64。
- 字符类型，char。
- 元组，当且仅当其包含的类型也都是 Copy 的时候。比如，(i32, i32) 是 Copy 的，但 (i32, String) 就不是。


In [4]:
let x = 5;
let y = x;

println!("x = {}, y = {}", x, y);

x = 5, y = 5


In [5]:
fn takes_ownership(some_string: String) { // some_string 进入作用域
    println!("{}", some_string);
} // 这里，some_string 移出作用域并调用 `drop` 方法。占用的内存被释放

fn makes_copy(some_integer: i32) { // some_integer 进入作用域
    println!("{}", some_integer);
} // 这里，some_integer 移出作用域。不会有特殊操作

let s = String::from("hello");  // s 进入作用域

takes_ownership(s);             // s 的值移动到函数里 ...
                                // ... 所以到这里不再有效

let x = 5;                      // x 进入作用域

makes_copy(x);                  // x 应该移动函数里，
                                // 但 i32 是 Copy 的，所以在后面可继续使用 x

hello
5


In [7]:
// 返回值也可以转移所有权
fn gives_ownership() -> String {             // gives_ownership 将返回值移动给
                                             // 调用它的函数

    let some_string = String::from("hello"); // some_string 进入作用域.

    some_string                              // 返回 some_string 并移出给调用的函数
}

// takes_and_gives_back 将传入字符串并返回该值
fn takes_and_gives_back(a_string: String) -> String { // a_string 进入作用域

    a_string  // 返回 a_string 并移出给调用的函数
}

let s1 = gives_ownership();         // gives_ownership 将返回值
                                        // 移给 s1

let s2 = String::from("hello");     // s2 进入作用域

let s3 = takes_and_gives_back(s2);  // s2 被移动到
                                    // takes_and_gives_back 中, 
                                    // 它也将返回值移给 s3
s3

"hello"

In [8]:
// 使用元组来返回多个值
fn main() {
    let s1 = String::from("hello");

    let (s2, len) = calculate_length(s1);

    println!("The length of '{}' is {}.", s2, len);
}

fn calculate_length(s: String) -> (String, usize) {
    let length = s.len(); // len() 返回字符串的长度

    (s, length)
}

main()

The length of 'hello' is 5.


()

In [10]:
fn main() {
    let s1 = String::from("hello");

    let len = calculate_length(&s1);

    println!("The length of '{}' is {}.", s1, len);
}

// 定义并使用一个（新的）calculate_length 函数，它以一个对象的引用作为参数而不是获取值的所有权
fn calculate_length(s: &String) -> usize {
    s.len()
}

main()

The length of 'hello' is 5.


()

In [12]:
// 尝试修改借用的值
// 正如变量默认是不可变的，引用也一样。（默认）不允许修改引用的值。

fn main() {
    let s = String::from("hello");

    change(&s);
}

fn change(some_string: &String) {
    some_string.push_str(", world");
}

Error: cannot borrow `*some_string` as mutable, as it is behind a `&` reference

In [14]:
// 必须将 s 改为 mut。然后必须创建一个可变引用 &mut s 和接受一个可变引用 some_string: &mut String

fn main() {
    let mut s = String::from("hello");

    change(&mut s);
}

fn change(some_string: &mut String) {
    some_string.push_str(", world");
}

In [7]:
// 不过可变引用有一个很大的限制：在特定作用域中的特定数据有且只有一个可变引用。这些代码会失败：

fn f(){
    let mut s = String::from("hello");

    let r1 = &mut s;
    let r2 = &mut s;

    println!("{}, {}", r1, r2);
}
f()

Error: cannot borrow `s` as mutable more than once at a time

这个限制的好处是 Rust 可以在编译时就避免数据竞争。数据竞争（data race）类似于竞态条件，它可由这三个行为造成：

- 两个或更多指针同时访问同一数据。
- 至少有一个指针被用来写入数据。
- 没有同步数据访问的机制。

数据竞争会导致未定义行为，难以在运行时追踪，并且难以诊断和修复；Rust 避免了这种情况的发生，因为它甚至不会编译存在数据竞争的代码！

In [6]:
fn f(){
    // 可以使用大括号来创建一个新的作用域，以允许拥有多个可变引用，只是不能 同时 拥有
    let mut s = String::from("hello");

    {
        let r1 = &mut s;

    } // r1 在这里离开了作用域，所以我们完全可以创建一个新的引用

    let r2 = &mut s;
    println!("{}", r2);
}
f()

hello


()

In [3]:
fn f(){
    let mut s = String::from("hello");

    let r1 = &s; // 没问题
    let r2 = &s; // 没问题
    println!("{} and {}", r1, r2);
    // 此位置之后 r1 和 r2 不再使用

    let r3 = &mut s; // 没问题
    println!("{}", r3);
}
f()

hello and hello
hello


()

In [8]:
fn no_dangle() -> String {
    let s = String::from("hello");

    s
}
no_dangle()

"hello"

In [10]:
fn first_word(s: &String) -> usize {
    let bytes = s.as_bytes();
    // enumerate 返回的元组中，第一个元素是索引，第二个元素是集合中元素的引用
    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return i;
        }
    }

    s.len()
}

let mut s = String::from("hello world");

let word = first_word(&s); // word 的值为 5

s.clear(); // 这清空了字符串，使其等于 ""
word

5

In [15]:
// 字符串 slice（string slice）是 String 中一部分值的引用，它看起来像这样：
fn f(){
    let s = String::from("hello world");

    let hello = &s[0..5];
    let world = &s[6..11];
    println!("{} - {}", hello, world);
}
f()

hello - world


()

In [17]:
fn fref(){
    let s = String::from("hello");

    let slice = &s[0..2];
    let slice = &s[..2];
    
    // 如果 slice 包含 String 的最后一个字节，也可以舍弃尾部的数字。这意味着如下也是相同的
    let s = String::from("hello");

    let len = s.len();

    let slice = &s[3..len];
    let slice = &s[3..];
    
    // 也可以同时舍弃这两个值来获取整个字符串的 slice。所以如下亦是相同的：

    let s = String::from("hello");

    let len = s.len();

    let slice = &s[0..len];
    let slice = &s[..];
}

fref()

()

In [20]:
fn first_word(s: &String) -> &str {
    let bytes = s.as_bytes();

    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return &s[0..i];
        }
    }

    &s[..]
}

fn main() {
    let mut s = String::from("hello world");

    let word = first_word(&s);

    // 借用规则，当拥有某值的不可变引用时，就不能再获取一个可变引用。
    // 因为 clear 需要清空 String，它尝试获取一个可变引用。Rust不允许这样做，因而编译失败
    // s.clear(); // 错误!

    println!("the first word is: {}", word);
}

main()

the first word is: hello


()

In [21]:
// 这里 s 的类型是 &str：它是一个指向二进制程序特定位置的 slice。这也就是为什么字符串字面值是不可变的；
// &str 是一个不可变引用。
let s = "Hello, world!";
s

"Hello, world!"

In [24]:
fn first_word(s: &str) -> &str {
    let bytes = s.as_bytes();

    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return &s[0..i];
        }
    }

    &s[..]
}
fn main() {
    let my_string = String::from("hello world");

    // first_word 中传入 `String` 的 slice
    let word = first_word(&my_string[..]);

    let my_string_literal = "hello world";

    // first_word 中传入字符串字面值的 slice
    let word = first_word(&my_string_literal[..]);

    // 因为字符串字面值 **就是** 字符串 slice，
    // 这样写也可以，即不使用 slice 语法！
    let word = first_word(my_string_literal);
}
main()

()

In [30]:
fn f(){
    let a = [1, 2, 3, 4, 5];

    let slice = &a[1..3];
    for s in slice.iter(){
        println!("{}", s);
    }
}
f()

2
3


()